In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import html
import numpy as np

In [5]:
df = pd.read_csv('../data_collecting/output/organicfood_withLoc.csv', lineterminator='\n')
print(len(df))
df.head()

145


,Unnamed: 0,tweetId,author_id_x,tweet,lang,created_at,topic_id,author_id_y,user_loc
0,0,1507507503336108032,2334273306,"Now That EVERYBODY Knows It, DC Finally Admits...",en,2022-03-25 23:59:31+00:00,633a60c58bc43c68579e,2334273306,NaN
1,1,1507507067577270272,826896534029479936,@RealOilRspctr @SeedOilDsrspctr I just saw thi...,en,2022-03-25 23:57:47+00:00,633a60c58bc43c68579e,826896534029479936,"Whidbey Island, WA"
2,2,1507505687911407616,631300815,Chicken Avon Analyst cross-platform Steel Spec...,en,2022-03-25 23:52:18+00:00,633a60c58bc43c68579e,631300815,Pontianak
3,3,1507503843101224962,815838530836500480,One teaspoon of soil contains more living orga...,en,2022-03-25 23:44:58+00:00,633a60c58bc43c68579e,815838530836500480,"Coimbatore,tamilnadu,India"
4,4,1507503584568840194,944520146411962368,So what was their marketing strategy? Genetica...,en,2022-03-25 23:43:57+00:00,633a60c58bc43c68579e,944520146411962368,The Blue Marble


In [6]:
df['lang'].unique()

array(['en', 'de', 'es'], dtype=object)